In [1]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
import main.packages.unchain_chain as chain
import main.packages.mine_generic as mine_g
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import mean_squared_error
from dieboldmariano import dm_test

In [2]:
def take_y_reals():
    cats = ["food", "energy", "neig", "services"]
    
    concatenated_df = pd.DataFrame()

    for cat in cats:
        cat_df = mine_g.load_excel(f"data/hicp_cat_raw/prc_hicp_{cat}.xlsx", name = f"hicp_{cat}", subset=True, verbose=0)
        #cat_df = cat_df[(cat_df.index > mine_g.train_test_split_date)]
        concatenated_df = pd.concat([concatenated_df, cat_df], axis=1)
    return concatenated_df

def extract_forecast_model_h(forecast_all_cat_df, model, h):
    """
    in 4 df, extract columns with model's name and horizon
    """

    pattern = rf"^{model}_(?!l).*h_{h}"
    matching_cols = [col for col in forecast_all_cat_df.columns if re.search(pattern, col)]
    df_specific_model_h = forecast_all_cat_df.loc[:, matching_cols]
    return df_specific_model_h

import os

def concatenate_csv_files(folder_path):
    """
    Concatenates CSV files in a folder column-wise into a single DataFrame.

    Args:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - concatenated_df (DataFrame): Concatenated DataFrame.
    """

    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    concatenated_df = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Add each file's data as new columns in the concatenated DataFrame
        concatenated_df = pd.concat([concatenated_df, df], axis=1)
        concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]

    return concatenated_df

def transform_to_chained(y_unchain, y_real):
    y_real = y_real[y_real.index <= "2022-12-31"]
    dec_mask = y_real.index.month == 12
    dec_data = y_real.where(dec_mask, other=np.nan)
    dec_data.ffill(inplace=True)
    dec_data = dec_data[dec_data.index > "2015-12-31"]
    # chain data back:
    y_chained = y_unchain * dec_data
    return y_chained

In [3]:
y_real = take_y_reals()


folder_path = "data/forecast_results"
forecast_yoy_all_cat = concatenate_csv_files(folder_path)
date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')

In [4]:
# take the weights:
weight_f = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_food.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_f = weight_f.iloc[1, :].to_frame().reset_index()
weight_f.columns = weight_f.iloc[0]
weight_f = weight_f[1:]
weight_f.rename(columns={"TIME": "date", "Germany": 'weight_f'}, inplace=True)

weight_e = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_energy.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_e = weight_e.iloc[1, :].to_frame().reset_index()
weight_e.columns = weight_e.iloc[0]
weight_e = weight_e[1:]
weight_e.rename(columns={"TIME": "date", "Germany": 'weight_e'}, inplace=True)

weight_n = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_neig.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_n = weight_n.iloc[1, :].to_frame().reset_index()
weight_n.columns = weight_n.iloc[0]
weight_n = weight_n[1:]
weight_n.rename(columns={"TIME": "date", "Germany": 'weight_n'}, inplace=True)

weight_s = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_services.xlsx', sheet_name='Sheet 1', skiprows=7)
weight_s = weight_s.iloc[1, :].to_frame().reset_index()
weight_s.columns = weight_s.iloc[0]
weight_s = weight_s[1:]
weight_s.rename(columns={"TIME": "date", "Germany": 'weight_s'}, inplace=True)

weights = pd.concat([weight_f, weight_e, weight_n, weight_s], axis=1)
weights = weights.loc[:, ~weights.columns.duplicated()]
weights['date'] = weights['date'].astype(int)
for col in weights.columns[1:]:
    weights[col] = weights[col].astype(float)

weights.loc[:, 'total'] = weights.iloc[:, 1:5].sum(axis=1)


/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's defaul

In [5]:
forecast_yoy_all_cat.columns

Index(['prc_food_h_1', 'prc_food_h_2', 'prc_food_h_3', 'xgb_food_h_1',
       'xgb_food_h_2', 'xgb_food_h_3', 'ols_food_h_1', 'ols_food_h_2',
       'ols_food_h_3', 'ridge_food_h_1', 'ridge_food_h_2', 'ridge_food_h_3',
       'lasso_food_h_1', 'lasso_food_h_2', 'lasso_food_h_3',
       'cb_ridge_prc_food_h_1', 'cb_ridge_prc_food_h_2',
       'cb_ridge_prc_food_h_3', 'cb_ridge_prc_lasso_food_h_1',
       'cb_ridge_prc_lasso_food_h_2', 'cb_ridge_prc_lasso_food_h_3',
       'cb_ridge_lasso_food_h_1', 'cb_ridge_lasso_food_h_2',
       'cb_ridge_lasso_food_h_3', 'prc_energy_h_1', 'prc_energy_h_2',
       'prc_energy_h_3', 'xgb_energy_h_1', 'xgb_energy_h_2', 'xgb_energy_h_3',
       'ols_energy_h_1', 'ols_energy_h_2', 'ols_energy_h_3',
       'ridge_energy_h_1', 'ridge_energy_h_2', 'ridge_energy_h_3',
       'lasso_energy_h_1', 'lasso_energy_h_2', 'lasso_energy_h_3',
       'cb_ridge_prc_energy_h_1', 'cb_ridge_prc_energy_h_2',
       'cb_ridge_prc_energy_h_3', 'cb_ridge_prc_lasso_energy_h_1'

In [13]:
new_df

,ols_food_h_3,ols_energy_h_3,ols_neig_h_3,ols_services_h_3
2016-01-31,1.772417,-6.639044,1.324065,2.828499
2016-02-29,1.929776,-5.707177,0.997705,1.173233
2016-03-31,1.537527,-4.823587,0.674918,1.370208
2016-04-30,1.378060,-4.165162,1.092497,1.244227
2016-05-31,1.296247,-6.528665,0.924912,1.100171
...,...,...,...,...
2022-08-31,7.182799,31.412410,4.673234,2.525408
2022-09-30,8.065042,31.307214,4.664659,1.656127
2022-10-31,9.322511,29.853745,4.821010,1.644452
2022-11-30,10.331021,30.047742,5.044471,1.643402


In [7]:
models = ['ols'] #'cb_ridge_prc', 'cb_ridge_prc_lasso', 'cb_ridge_lasso'
horizons = [1, 2, 3]

for model in models:
    for h in horizons:

        # get the categorical data:
        new_df = extract_forecast_model_h(forecast_yoy_all_cat, model, h)
        new_df.set_index(date_range, inplace=True)

        # transform:
        food_chained = chain.transform_back_chained(new_df.iloc[:, 0], y_real.iloc[:, 0])
        food_unchain = chain.unchain_series(food_chained, y_real.iloc[:, 0])
        energy_chained = chain.transform_back_chained(new_df.iloc[:, 1], y_real.iloc[:, 1])
        energy_unchain = chain.unchain_series(energy_chained, y_real.iloc[:, 1])
        neig_chained = chain.transform_back_chained(new_df.iloc[:, 2], y_real.iloc[:, 2])
        neig_unchained = chain.unchain_series(neig_chained, y_real.iloc[:, 2])
        services_chained = chain.transform_back_chained(new_df.iloc[:, 3], y_real.iloc[:, 3])
        services_unchained = chain.unchain_series(services_chained, y_real.iloc[:, 3])
        
        # take the products of respective category and weights
        food_unchain.name = 'food_unchain'
        food_unchain = food_unchain.to_frame()
        food_unchain.loc[:, 'year'] = food_unchain.index.year
        food_unchain
        merged_df = pd.merge(food_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_food = merged_df.loc[:, 'food_unchain'].mul(merged_df.loc[:, 'weight_f'])

        energy_unchain.name = 'e_unchain'
        e_unchain = energy_unchain.to_frame()
        e_unchain.loc[:, 'year'] = e_unchain.index.year
        merged_df = pd.merge(e_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_e = merged_df.loc[:, 'e_unchain'].mul(merged_df.loc[:, 'weight_e'])


        neig_unchained.name = 'n_unchain'
        n_unchain = neig_unchained.to_frame()
        n_unchain.loc[:, 'year'] = n_unchain.index.year
        merged_df = pd.merge(n_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_n = merged_df.loc[:, 'n_unchain'].mul(merged_df.loc[:, 'weight_n'])

        services_unchained.name = 's_unchain'
        s_unchain = services_unchained.to_frame()
        s_unchain.loc[:, 'year'] = s_unchain.index.year
        merged_df = pd.merge(s_unchain, weights, how='inner', left_on='year', right_on='date')
        mul_s = merged_df.loc[:, 's_unchain'].mul(merged_df.loc[:, 'weight_s'])

        # take the sum of all product:
        mul_sum = pd.concat([mul_food, mul_e, mul_n, mul_s], axis=1)
        mul_sum.loc[:, 'hicp_unchained'] = mul_sum.sum(axis=1)/1000

        date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')
        mul_sum.set_index(date_range, inplace=True)

        # chain the results:
        y_real_hicp = mine_g.load_excel('data/hicp_all.xlsx', 'hicp_all', verbose=0)
        hicp_chained_final = transform_to_chained(mul_sum.loc[:, 'hicp_unchained'], y_real_hicp.iloc[:, 0])

        y_real_hicp.loc[:, 'last_y'] = y_real_hicp.iloc[:, 0].shift(12)
        y_real_hicp = y_real_hicp[(y_real_hicp.index > mine_g.train_test_split_date) & (y_real_hicp.index <= mine_g.max_X_date)]

        hicp_all_yoy = (hicp_chained_final/y_real_hicp.loc[:, 'last_y']-1)*100
        hicp_all_yoy.name = f'{model}_h_{h}'   
        forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')
        new_forecast = pd.concat([forecast, hicp_all_yoy.reset_index(drop=True)], axis=1)


        #new_forecast.to_csv('data/headline_forecast/head_infl_forecast.csv', index=False)

----

# Compare OLS for headline and OLS from individual cat aggregated

In [8]:
forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')

real_hicp_yoy = pd.read_csv('data/preprocessed/head_inflation.csv', parse_dates = True, index_col='date')
real_hicp_yoy_test = real_hicp_yoy[(real_hicp_yoy.index > mine_g.train_test_split_date) & (real_hicp_yoy.index <= mine_g.max_X_date)]


In [9]:
forecast

,ar_110_h_1,ar_110_h_2,ar_110_h_3,ar_ols_h_1,ar_ols_h_2,ar_ols_h_3,prc_h_1,prc_h_2,prc_h_3,xgb_h_1,...,lasso_h_3,cb_ridge_prc_h_1,cb_ridge_prc_h_2,cb_ridge_prc_h_3,cb_ridge_prc_lasso_h_1,cb_ridge_prc_lasso_h_2,cb_ridge_prc_lasso_h_3,cb_ridge_lasso_h_1,cb_ridge_lasso_h_2,cb_ridge_lasso_h_3
0,0.201236,0.329779,1.160298,0.305073,0.400090,1.253522,0.477723,0.879217,1.155731,0.720385,...,0.839818,0.645218,0.792337,0.866038,0.724769,0.784596,0.857298,0.848292,0.737285,0.708082
1,0.377993,0.201232,0.333175,0.493679,0.399921,0.486686,0.187900,0.325666,0.682909,0.703246,...,0.865850,0.391260,0.517978,0.662904,0.459010,0.591486,0.730553,0.594565,0.724396,0.754375
2,-0.025772,0.382279,0.201233,0.029098,0.572149,0.486532,-0.262701,0.057059,0.181945,0.345818,...,0.768718,-0.100215,0.192532,0.351504,-0.043260,0.242019,0.490575,0.066460,0.334499,0.644890
3,0.070035,-0.036898,0.381662,0.212929,0.147909,0.643806,-0.134229,-0.315474,0.018045,0.273920,...,0.520427,-0.055320,-0.141534,0.125177,-0.009916,-0.047416,0.256927,0.052240,0.086612,0.376367
4,-0.238173,0.074585,-0.035253,-0.152066,0.315777,0.256404,-0.189562,-0.133276,-0.265374,-0.057299,...,0.033996,-0.210853,-0.186746,-0.252147,-0.254219,-0.188029,-0.156766,-0.286548,-0.215405,-0.102462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,8.453284,8.295415,8.659386,7.862867,7.111033,6.988278,8.789301,8.881570,9.070468,6.766554,...,9.353103,9.031287,9.318295,9.625460,9.039907,9.303563,9.534674,9.165210,9.514559,9.766777
80,8.796783,8.455887,8.295984,8.188482,7.301452,6.614902,9.531649,9.304273,9.162557,6.921153,...,9.498422,9.456552,9.534618,9.740163,9.418017,9.477484,9.659582,9.361200,9.564090,9.908095
81,10.712508,8.800719,8.455609,10.074116,7.598793,6.788786,11.255497,10.205313,9.737996,7.790400,...,10.695643,11.722368,10.860489,10.653532,11.664883,10.975143,10.667569,11.869576,11.360058,11.132356
82,11.523992,10.729384,8.800305,10.680094,9.320693,7.060308,11.863813,11.666736,10.465141,8.762138,...,11.908712,12.102763,12.210797,11.446058,11.911386,12.228984,11.600276,11.935173,12.510107,12.167844


In [10]:
rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast[h_cols]
    
    for col in [ col for col in forecast_df_h.columns if "ar_ols" not in col]:
        print(col)
        print(dm_test(real_hicp_yoy_test.iloc[:, 0], forecast_df_h[col], forecast_df_h.loc[:, f'ar_ols_h_{h}'], h = h, harvey_correction=True))


    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], real_hicp_yoy_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]
rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, int(len(forecast.columns)/3)), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])


ar_110_h_1
(-1.6769896095002441, 0.0973070531150434)
prc_h_1
(-1.5781254467216208, 0.11834067278353033)
xgb_h_1
(3.4237065320769067, 0.0009619323061758958)
ols_h_1
(3.539041479399328, 0.0006609258505795756)
ridge_h_1
(-1.0653307344634626, 0.289815590386732)
lasso_h_1
(-0.8946303986000829, 0.37357171115027277)
cb_ridge_prc_h_1
(-1.6289865490960187, 0.10710576840526909)
cb_ridge_prc_lasso_h_1
(-1.7453848181394982, 0.08461869679593281)
cb_ridge_lasso_h_1
(-1.3974531417190776, 0.16600170813092985)
ar_110_h_2
(-1.5284227334846807, 0.13020984156895388)
prc_h_2
(-1.2279405052947059, 0.22294023474817415)
xgb_h_2
(2.3304612418639303, 0.022209175970383455)
ols_h_2
(1.724296670959621, 0.08837689553732869)
ridge_h_2
(-1.4989486791627376, 0.13768095075089137)
lasso_h_2
(-1.505347480195841, 0.13603101380567806)
cb_ridge_prc_h_2
(-1.4854956017988998, 0.14120103861820776)
cb_ridge_prc_lasso_h_2
(-1.530942585813958, 0.12958626261347939)
cb_ridge_lasso_h_2
(-1.5466068948854563, 0.1257626711967298)
ar_11

In [11]:
rmse

,ar_110,ar_ols,prc,xgb,ols,ridge,lasso,cb_ridge_prc,cb_ridge_prc_lasso,cb_ridge_lasso
h_1,0.626385,0.696370,0.589798,1.154790,1.527996,0.628242,0.644231,0.590541,0.592990,0.616740
h_2,0.883125,1.075871,0.850622,1.423182,1.499872,0.750642,0.779543,0.772864,0.767240,0.754751
h_3,1.116769,1.431566,1.058617,1.484928,1.657973,0.856219,0.929474,0.920781,0.917174,0.881245


In [72]:
rmse.to_csv("data/report_rmse/total.csv")

In [12]:
rmse_comparative = [round(rmse[col]/rmse[f'ar_ols'], 2) for col in rmse.columns]
rmse_comparative

[h_1    0.90
 h_2    0.82
 h_3    0.78
 dtype: float64,
 h_1    1.0
 h_2    1.0
 h_3    1.0
 Name: ar_ols, dtype: float64,
 h_1    0.85
 h_2    0.79
 h_3    0.74
 dtype: float64,
 h_1    1.66
 h_2    1.32
 h_3    1.04
 dtype: float64,
 h_1    2.19
 h_2    1.39
 h_3    1.16
 dtype: float64,
 h_1    0.9
 h_2    0.7
 h_3    0.6
 dtype: float64,
 h_1    0.93
 h_2    0.72
 h_3    0.65
 dtype: float64,
 h_1    0.85
 h_2    0.72
 h_3    0.64
 dtype: float64,
 h_1    0.85
 h_2    0.71
 h_3    0.64
 dtype: float64,
 h_1    0.89
 h_2    0.70
 h_3    0.62
 dtype: float64]